In [ ]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import save_model, load_model
import os
from dotenv import load_dotenv

load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")

def fetch_data_from_mongodb():
    client = MongoClient(MONGO_URI)
    db = client["water_level_db"]
    collection = db["water_level_data"]
    return pd.DataFrame(list(collection.find()))

def preprocess_data(dwlr_data):
    dwlr_data["date"] = pd.to_datetime(dwlr_data["date"])
    dwlr_data = dwlr_data.sort_values("date")
    
    # Scale all features
    scaler_dict = {}
    features = ['water_level', 'temperature', 'rainfall']
    scaled_features = {}
    
    for feature in features:
        scaler = MinMaxScaler()
        scaled_features[feature] = scaler.fit_transform(dwlr_data[feature].values.reshape(-1, 1))
        scaler_dict[feature] = scaler
    
    for feature, scaled_values in scaled_features.items():
        dwlr_data[f'scaled_{feature}'] = scaled_values
        
    return dwlr_data, scaler_dict

def prepare_lstm_data(data, window_size):
    features = ['scaled_water_level', 'scaled_temperature', 'scaled_rainfall']
    X, y = [], []
    
    for i in range(len(data) - window_size):
        # Stack all features for each window
        features_window = np.column_stack([data[feature][i:i + window_size] for feature in features])
        X.append(features_window)
        y.append(data['scaled_water_level'].iloc[i + window_size])
    
    return np.array(X), np.array(y)

def build_lstm_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(50, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(50),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    return model

def train_and_save_models():
    data = fetch_data_from_mongodb()
    window_size = 10
    models_info = {}

    # Create models directory if it doesn't exist
    os.makedirs('models', exist_ok=True)

    for dwlr_id in data["dwlr_id"].unique():
        dwlr_data = data[data["dwlr_id"] == dwlr_id]
        dwlr_data, scalers = preprocess_data(dwlr_data)
        
        X, y = prepare_lstm_data(dwlr_data, window_size)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)
        
        # Save model and scalers with correct extension
        model_path = f'models/model_{dwlr_id}.keras'
        scalers_path = f'models/scalers_{dwlr_id}.pkl'
        
        model.save(model_path)
        with open(scalers_path, 'wb') as f:
            pickle.dump(scalers, f)
            
        rolling_mean = dwlr_data["water_level"].rolling(window=10).mean().iloc[-1]
        rolling_std = dwlr_data["water_level"].rolling(window=10).std().iloc[-1]
        
        models_info[dwlr_id] = {
            'model_path': model_path,
            'scalers_path': scalers_path,
            'rolling_mean': rolling_mean,
            'rolling_std': rolling_std,
            'window_size': window_size
        }
    
    with open('models/models_info.pkl', 'wb') as f:
        pickle.dump(models_info, f)

if __name__ == "__main__":
    train_and_save_models()



Epoch 1/10


C:\Users\PRADIP\AppData\Local\Temp\ipykernel_12712\1813772480.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dwlr_data["date"] = pd.to_datetime(dwlr_data["date"])
C:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 0.2648 - val_loss: 0.1512
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1056 - val_loss: 0.0946
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0834 - val_loss: 0.1054
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0838 - val_loss: 0.0953
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0861 - val_loss: 0.1001
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0858 - val_loss: 0.0978
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0759 - val_loss: 0.0985
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0779 - val_loss: 0.0980
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0831 - val_loss: 0.0956
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0840 - val_loss: 0.0983
Epoch 1/10


C:\Users\PRADIP\AppData\Local\Temp\ipykernel_12712\1813772480.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dwlr_data["date"] = pd.to_datetime(dwlr_data["date"])
C:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.2628 - val_loss: 0.1123
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0991 - val_loss: 0.0906
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0878 - val_loss: 0.0887
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0920 - val_loss: 0.0879
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0875 - val_loss: 0.0881
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0894 - val_loss: 0.0878
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0858 - val_loss: 0.0881
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0887 - val_loss: 0.0877
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0891 - val_loss: 0.0888
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0891 - val_loss: 0.0876
Epoch 1/10


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.2057 - val_loss: 0.1132
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0954 - val_loss: 0.0928
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0841 - val_loss: 0.0895
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0952 - val_loss: 0.0880
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0860 - val_loss: 0.0855
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0842 - val_loss: 0.0853
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0840 - val_loss: 0.0860
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0839 - val_loss: 0.0853
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0840 - val_loss: 0.0857
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0796 - val_loss: 0.0847
Epoch 1/10


C:\Users\PRADIP\AppData\Local\Temp\ipykernel_12712\1813772480.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dwlr_data["date"] = pd.to_datetime(dwlr_data["date"])
C:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.2371 - val_loss: 0.0990
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1000 - val_loss: 0.0838
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0993 - val_loss: 0.0807
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0868 - val_loss: 0.0808
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0886 - val_loss: 0.0808
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0955 - val_loss: 0.0807
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0865 - val_loss: 0.0804
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0843 - val_loss: 0.0824
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0900 - val_loss: 0.0809
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0863 - val_loss: 0.0805
Epoch 1/10


C:\Users\PRADIP\AppData\Local\Temp\ipykernel_12712\1813772480.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dwlr_data["date"] = pd.to_datetime(dwlr_data["date"])
C:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.2251 - val_loss: 0.0886
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1042 - val_loss: 0.0916
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0916 - val_loss: 0.0793
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0908 - val_loss: 0.0852
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0878 - val_loss: 0.0797
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0828 - val_loss: 0.0838
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0887 - val_loss: 0.0804
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0827 - val_loss: 0.0824
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0900 - val_loss: 0.0802
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0886 - val_loss: 0.0813
